원본: https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html#sphx-glr-beginner-nlp-advanced-tutorial-py

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dynamic-versus-Static-Deep-Learning-Toolkits" data-toc-modified-id="Dynamic-versus-Static-Deep-Learning-Toolkits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dynamic versus Static Deep Learning Toolkits</a></span></li><li><span><a href="#Bi-LSTM-Conditional-Random-Field-Discussion" data-toc-modified-id="Bi-LSTM-Conditional-Random-Field-Discussion-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Bi-LSTM Conditional Random Field Discussion</a></span></li><li><span><a href="#Implementation-Notes" data-toc-modified-id="Implementation-Notes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Implementation Notes</a></span></li></ul></div>

##  Dynamic versus Static Deep Learning Toolkits

Pytorch는 dynamic neural network를 수행하기 위한 도구이다. 

다른 도구로는 Dynet이 있다.(Pytorch와 Dynet이 유사하게 작동되기 때문에 Dynet에 예제가 있다면 Pytorch에서 구현하는데 도움이 될 것이다.)

반대로 Static tool kit을 수행하는 TensorFlow, Keras, Theano등이 있다. 

핵심 차이점은 아래와 같다.

static toolkit에서는 계산 그래프를 정의하고, 컴파일 한 다음, instance를 stream합니다.

하지만 dynamic toolkit에서는 각 instance에 대해 계산 그래프를 정의합니다. 컴파일 되지 않고 즉시 실행됩니다.

많은 경험이 있지 않다면, 이러한 차이를 이해하기가 어려울 수 있습니다.

한 가지의 예로 모델이 deep하게 구성된 parser를 build 한다고 가정하는 것입니다.

모델에 다음 단계가 수반되어진다고 가정합니다.

tree의 뿌리를 구축

root nodes에 Tag를 지정(문장의 단어)

이제 여기에서 신경망과 임베딩을 사용하여 구성 요소를 형성하는 조합을 찾아야 합니다.

새로운 구성 요소를 찾을 때마다, 몇 가지의 테크닉을 사용하여 구성 요소의 임베딩을 얻어야 합니다.

이 경우에, 우리의 네트워크 구조는 전적으로 input sentence에 의존하게 될 것입니다.

문장 안에서 'The green cat scratched the wall'은 모델의 어느 한 부분이고, 
span(i, j, r) = (1, 3, NP) (즉, NP 구성 요소의 spans word 1에서  word 3까지, 이 경우"The green cat").

그러나 또 다른 문장은 "Somewhere, the big fat cat scratched the wall"이 될 수도 있다.

문장 안에서, 구성요소가 (2, 4, NP)가 되기를 원합니다. 이 구성요소는 instace에 의존할 것입니다.

만약 static toolkit처럼 계산 그래프를 한 번만 컴파일 하면, 이 logic를 프로그래밍 하는 것이 매우 어렵거나 불가능합니다.

하지만 dynamic toolkit에서 계산 그래프를 미리 정의할 필요가 없습니다.

각 인스턴스마다 새로운 계산 그래프가 있을 수 있으므로, 문제를 해결할 수 있습니다.

dynamic toolit은 디버깅하기도 쉽고, 코드 자체도 Host언어와 비슷하다는 장점이 있습니다.
(Pytorch와 Dynet이 Keras 또는 Theano보다 실제 Python 코드와 비슷하다는 것을 의미합니다).

## Bi-LSTM Conditional Random Field Discussion

이번 섹션에서는, 명명된 entity 인식을 위한 Bi-LSTM 조건부 임의 필드의 전체적이고 복잡한 예를 볼 것입니다.

위의 LSTM tagger는 일반적으로 품사 태깅에 충분하지만, CRF와 같은 시퀸스 모델들은 NER(개체명 인식)의 강력한 성능이 필수적입니다.

CRF에 익숙하다고 가정을 합니다.

## Implementation Notes

아래의 예제는 partition function을 계산하기 위해 로그 공간에 순방향 알고리즘을 구현하고 디코딩 할 viterbi 알고리즘을 구현합니다.

역전파는 자동으로 계산되어집니다. 우리가 직접 계산할 필요가 없다.

구현이 최적화되지 않았습니다. 

만약 무슨일이 일어나는지 이해했다면, 순방향 알고리즘내에서 다음 태그를 반복하는 것이 한 번의 큰 작업으로 이루어진다는 것을 알 수 있습니다.

더 읽기 쉽게 코딩하고 싶었습니다.

만약 너가 관련된 사항을 변경하기를 원한다면, 실제 작업에 이 태그를 아마도 사용해야 할 것입니다.

In [1]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Helper functions들은 코드를 더 쉽게 읽도록 도와줍니다.

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [3]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)  # tensor([[-10000., -10000., -10000., -10000., -10000.]])
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0. # tensor([[-10000., -10000., -10000.,      0., -10000.]])

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [4]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!

(tensor(2.6907), [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
(tensor(20.4906), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
